This Python Jupyter Notebook creates the intial information for use within the UK General Elections model

In [1]:
#Import required packages
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import pandas as pd

In [2]:
#Connect to database 'UK_General_Election' using SQlAlchemy
connection_str = "DRIVER={SQL SERVER};SERVER=DANZPOOTA;DATABASE=UK_General_Election;TRUSTED_CONNECTION=YES"
params = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
conn = engine.connect()

In [3]:
DeleteExistingData = """DELETE PollDetails
DELETE PollMeta"""
engine.execute(DeleteExistingData)

In [4]:
#Location of file to be used
T22Xls = "C:\\Users\\danmu\\Documents\\Elections\\2024_Python\\UK_Polls_Import_2022.xlsx"
LondonXls = "C:\\Users\\danmu\\Documents\\Elections\\2024_Python\\London_Polls_Import.xlsx"
WalesXls = "C:\\Users\\danmu\\Documents\\Elections\\2024_Python\\Wales_Polls_Import.xlsx"

In [5]:
T22_meta_df = pd.read_excel(T22Xls, sheet_name='PollMeta')
London_meta_df = pd.read_excel(LondonXls, sheet_name='PollMeta')
Wales_meta_df = pd.read_excel(WalesXls, sheet_name='PollMeta')
frames = [T22_meta_df, London_meta_df, Wales_meta_df]
meta_df = pd.concat(frames,ignore_index=True)
meta_df.head()

,PollDate,Pollster,SampleSize,PollType,PollScope,PollScopeAll,PollScopeRegion,PollScopeConst,PollLink
0,2022-12-28,PeoplePolling,1169,YouGov,All,1,NaN,NaN,https://peoplepolling.org/tables/202212_GBN_W5...
1,2022-12-22,Omnisis,1243,ITL1Region,All,1,NaN,NaN,https://omnisis.co.uk/media/1200/vi-015-tabs-2...
2,2022-12-22,Techne,1633,GB,All,1,NaN,NaN,https://www.techneuk.com/wp-content/uploads/20...
3,2022-12-21,YouGov,1672,YouGov,All,1,NaN,NaN,https://docs.cdn.yougov.com/53o943qhjj/TheTime...
4,2022-12-18,Savanta,2024,ITL1Region,All,1,NaN,NaN,https://savanta.com/wp-content/uploads/2023/01...


In [6]:
T22_details_df = pd.read_excel(T22Xls, sheet_name='PollDetails')
London_details_df = pd.read_excel(LondonXls, sheet_name='PollDetails')
Wales_details_df = pd.read_excel(WalesXls, sheet_name='PollDetails')
frames = [T22_details_df, London_details_df, Wales_details_df]
details_df = pd.concat(frames,ignore_index=True)
details_df.head()

,PollID,RegionName,Party,VoteShare
0,20221201BMGITL1RegionAll,East Midlands,Con,0.308258
1,20221201BMGITL1RegionAll,East Midlands,Lab,0.533354
2,20221201BMGITL1RegionAll,East Midlands,LD,0.030100
3,20221201BMGITL1RegionAll,East Midlands,PC,0.000000
4,20221201BMGITL1RegionAll,East Midlands,SNP,0.000000


In [7]:
#Insert into database then read back out into a dataframe to check it has been inserted
meta_df.to_sql('PollMeta', conn, if_exists='append', index=False)
db_meta_df = pd.read_sql('SELECT * From PollMeta',conn)
db_meta_df.head()

,PollID,Pollster,PollType,PollDate,PollScope,PollScopeAll,PollScopeRegion,PollScopeConst,SampleSize,PollLink
0,20200126YouGovNationWales,YouGov,Nation,2020-01-26,Wales,False,Wales,None,1037,https://www.itv.com/news/wales/2020-01-29/poll...
1,20200306YouGovLondonTwoAll,YouGov,LondonTwo,2020-03-06,All,True,None,None,1002,https://docs.cdn.yougov.com/t1teszt721/QMUL_Lo...
2,20200407YouGovWalesRegionAll,YouGov,WalesRegion,2020-04-07,All,True,None,None,1008,https://docs.cdn.yougov.com/kg4inoaeii/Results...
3,20200601YouGovWalesRegionAll,YouGov,WalesRegion,2020-06-01,All,True,None,None,1021,https://docs.cdn.yougov.com/6a5jrgxa0c/Results...
4,20200807Redfield & WiltonLondonTwoAll,Redfield & Wilton,LondonTwo,2020-08-07,All,True,None,None,2500,https://redfieldandwiltonstrategies.com/wp-con...


In [8]:
#Insert into database then read back out into a dataframe to check it has been inserted
details_df.to_sql('PollDetails', conn, if_exists='append', index=False)
db_details_df = pd.read_sql('SELECT * From PollDetails',conn)
db_details_df.head()

,PollDetailsID,PollID,RegionName,Party,VoteShare
0,20200126YouGovNationWalesWalesCon,20200126YouGovNationWales,Wales,Con,0.41
1,20200126YouGovNationWalesWalesGreen,20200126YouGovNationWales,Wales,Green,0.02
2,20200126YouGovNationWalesWalesLab,20200126YouGovNationWales,Wales,Lab,0.36
3,20200126YouGovNationWalesWalesLD,20200126YouGovNationWales,Wales,LD,0.05
4,20200126YouGovNationWalesWalesOther,20200126YouGovNationWales,Wales,Other,0.01


In [9]:
conn.close()